In [1]:
import babelnet as bn
from babelnet import BabelSynsetID, Language
from babelnet.data.relation import BabelPointer

2023-09-02 15:31:11,627 [babelnet.conf] INFO: Loaded configuration from ['/home/giovanni/unimore/2_graph_analytics/tesina/babeldist/remote/babelnet_conf.yml']
2023-09-02 15:31:11,638 [babelnet.api] INFO: BabelNet Remote Procedure Call API v1.1.0


In [2]:
bn.version()

5.0

In [5]:
import sys
import time
sys.path.append('../')
from utils import utils
from zerorpc import TimeoutExpired, LostRemote

In [4]:
### Root searcher
visited = set()

max_visits, n = 10, 0
start_synset_id = 'bn:17335263n' # Pandora (gioielli)
start_synset = bn.get_synset(BabelSynsetID(start_synset_id)) # a volte dà LostRemote anche qua...
q = [start_synset]
fname = utils.get_current_logfile_number('candidate_roots', extension='.csv')

with open(fname, 'w') as logfname:
    start_t = time.time()
    while q and n < max_visits:
        synset = q.pop(0)
        n += 1
        pb = utils.get_progress_bar(int((n / max_visits) * 100))
        print(pb, end='\r')

        hypernym_edges = []
        
        try:
            hypernym_edges = synset.outgoing_edges(BabelPointer.ANY_HYPERNYM)        
            if hypernym_edges == []:
                # print(f'Possible root node: {synset.id}')
                logfname.write(f'{synset.id},\
                    candidate_root,\
                    {synset.main_sense().full_lemma},\
                    {len(synset.outgoing_edges())},\
                    {len(synset.outgoing_edges(BabelPointer.ANY_HYPERNYM))},\
                    {synset.sense_sources}'.replace(' ', '') +'\n')
        except (TimeoutExpired, LostRemote) as e:
            logfname.write(f'{synset.id},{utils.get_full_class_name(e)},{None},{None},{None}\n')
        
        for edge in hypernym_edges:
            if edge.id_target not in visited:
                try:
                    q.append(bn.get_synset(edge.id_target))
                    visited.add(edge.id_target)
                except (TimeoutExpired, LostRemote) as e:
                    logfname.write(f'{synset.id},{utils.get_full_class_name(e)},{None},{None},{None}\n')                
    
    logfname.write(f'start_node={start_synset_id}\n')
    logfname.write(f'max_visits={max_visits}\n')
    if q == []: logfname.write('Queue empy\n')
    if n == max_visits: logfname.write('Reached max visits\n')
    end_t = time.time()
    min, sec = divmod(end_t - start_t, 60)
    logfname.write(f'total_time,{int(min)}m,{int(sec)}s')

Sembra che i synset con una qualche sorgente WIKI* non abbiano iperonimi anche se non sono proprio dei concetti
basilari, come "Religious_calling" o "engineering_process".
Inoltre il server dà qualche problema e non riesce a recuperare sempre tutto in tempi brevi, ad esempio
con "homo" va sempre in palla e con "taxonomic_group" quasi sempre, nel file di log "candidate_roots_sources(1).csv" ad esempio si vede che sono stati trovati 1954260 link ad altri synset di cui 2 iperonimi, ma questo indica
anche che nella ricerca delle radici il server aveva detto che ne aveva 0, una possibile risposta quando va
oltre il tempo massimo o situazioni di questo genere forse.

Più è elevato il numero di edge da ricercare più è difficile che restituisca il numero corretto, di solito ritorna 0 o si scatena un errore LostRemote/TimeoutExpired, ma anche con numeri inferiori a volte fallisce, con
"field_of_study" non riesce a ritornare il risultato corretto benché abbia solo 77 edge totali.

In [6]:
import pandas as pd

In [13]:
cr0 = pd.read_csv('../log/candidate_roots.csv', sep=';', skipfooter=4, engine='python', \
    names=['synsetID', 'type', 'lemma', 'num_out_edges', 'num_hypernyms', 'sense_sources'])
cr0

,synsetID,type,lemma,num_out_edges,num_hypernyms,sense_sources
0,bn:00044576n,candidate_root,homo,0,0,"[OMWN_FA,WN,WNTR,OMWN_NO,OMWN_JA,OMWN_HE,OMWN_..."
1,bn:00064608n,candidate_root,profession,0,0,"[OMWN_EL,WNTR,OMWN_DA,OMWN_NN,OMWN_NO,OMWN_JA,..."
2,bn:04751152n,candidate_root,production_center,7,0,"[WIKIDATA_ALIAS,WIKIDATA,WIKIRED,WIKI]"
3,bn:00076248n,candidate_root,taxonomic_group,0,0,"[OMWN_EL,WNTR,WN,OMWN_PL,OMWN_JA,OMWN_TH,WN202..."
4,bn:06429342n,candidate_root,Dokument_normatywny,5,0,"[WIKI,WIKIDATA]"
5,bn:00248578n,candidate_root,Trade_literature,22,0,"[WIKI,WIKIRED,WIKIDATA]"
6,bn:00031027n,candidate_root,entity,18,0,"[OMWN_EL,WNTR,WN,OMWN_TH,OMWN_HE,OMWN_ZH,OMWN_..."
7,bn:15921369n,candidate_root,adaptation,14,0,"[WIKIDATA,WIKI]"
8,bn:00894769n,candidate_root,task_(computing),40,0,"[WIKIDATA,WIKITR,WIKIDATA_ALIAS,WIKI,WIKIRED,W..."
9,bn:14481752n,candidate_root,field_of_study,77,0,"[WIKIDATA,WIKIDATA_ALIAS,WIKIRED,WIKI]"


In [ ]:
bn.sense